<a href="https://colab.research.google.com/github/malraharsh/Pruning-Experiments/blob/master/Pruning_Experiments_on_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q tensorflow-model-optimization

import tempfile
import os

import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
import pandas as pd


from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from tensorflow import keras

# %load_ext tensorboard

if not os.path.exists('log'):
    os.mkdir('log')

import tensorflow_model_optimization as tfmot


from IPython.display import display

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

SHOW = False

In [ ]:
data = datasets.cifar10.load_data()

(Train_images, Train_labels), (Test_images, Test_labels) = data

# pct_data = 0.1
# top = int(np.ceil(Train_images.shape[0] * pct_data))

# (train_images, train_labels), (test_images, test_labels) = (Train_images[:top], Train_labels[:top]), (Test_images[:top], Test_labels[:top])
# print(top)

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [3]:
train_images = train_labels = test_images = test_labels = None

def change_pct_data(pct_data):    
    global train_images, train_labels, test_images, test_labels    
    top = int(np.ceil(Train_images.shape[0] * pct_data))
    (train_images, train_labels), (test_images, test_labels) = (Train_images[:top], Train_labels[:top]), (Test_images[:top], Test_labels[:top])
    print(f"No of data - {top}")

In [ ]:
def train(train_images, train_labels, test_images, test_labels):

    # Normalize the input image so that each pixel value is between 0 to 1.
    train_images = train_images.copy() / 255.0 #!!!!! CAN REOMVE COPY
    test_images = test_images.copy() / 255.0

    # Define the model architecture.
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10))

    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    history = model.fit(train_images, train_labels, epochs=EPOCHS, 
                        validation_data=(test_images, test_labels), verbose=VERBOSE)

    #no test, only val data
    # test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)
    # history.history['test_loss'] = [test_loss]
    # history.history['test_accuracy'] = [test_accuracy]

    if SHOW:

        plt.plot(history.history['accuracy'], label='accuracy')
        plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.ylim([0.5, 1])
        plt.legend(loc='lower right')

        # test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

    
    return model, history.history
    

def prune(train_images, train_labels, model):

    # Compute end step to finish pruning after 2 epochs.
    batch_size = 128
    epochs = EPOCHS_PRUNE
    validation_split = 0.1 # 10% of training set will be used for validation set. 

    num_images = train_images.shape[0] * (1 - validation_split)
    end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

    # Define model for pruning.
    pruning_params = {
          'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                                   final_sparsity=0.80,
                                                                   begin_step=0,
                                                                   end_step=end_step)
    }

    model_for_pruning = prune_low_magnitude(model, **pruning_params)

    # `prune_low_magnitude` requires a recompile.
    model_for_pruning.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    # model_for_pruning.summary()

    callbacks = [
      tfmot.sparsity.keras.UpdatePruningStep(),
    #   tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
    ]

    history = model_for_pruning.fit(train_images, train_labels,
                      batch_size=batch_size, epochs=epochs, validation_data=(test_images, test_labels),
                      callbacks=callbacks, verbose=VERBOSE)
    
    # test_loss, test_accuracy = model_for_pruning.evaluate(test_images, test_labels, verbose=0)
    # history.history['test_loss'] = [test_loss]
    # history.history['test_accuracy'] = [test_accuracy]

    if SHOW:
        print('Pruned test accuracy:', test_accuracy)

    return model_for_pruning, history.history

In [ ]:
def do_train(test_pct):
    train_x, val_x, train_y, val_y = train_test_split(train_images, train_labels, test_size=test_pct, stratify=train_labels)
    
    print("TRAINING ---")
    model, info_train = train(train_x, train_y, val_x, val_y)
    
    print("PRUNING ---")
    _, info_prune = prune(train_x, train_y, model)
    
    return info_train, info_prune

In [ ]:
def add_info(dic, pct_train, info):
    # print(list(dic.items()), '----')
    dic = {k:v[-1] for k, v in dic.items()}
    dic['percentage'] = pct_train
    return info.append(dic, ignore_index=True)   

def save(df, name, pct_data):
    df.to_csv(f'log/info-{name}-{pct_data*100}%.csv')


# def savefile():
#     df_info_train = add_info(info_train, 1 - pct_test, df_info_train)
#     df_info_prune = add_info(info_prune, 1 - pct_test, df_info_prune)

#     save(df_info_train, 'train', pct_data)
#     save(df_info_prune, 'prune', pct_data)


def full_pct_data(pct_data, pct_test=0.2): #pct of full data
    change_pct_data(pct_data) 
    df_info_train = pd.DataFrame()
    df_info_prune = pd.DataFrame()

    print(f'\n Percentage of Whole data {pct_data*100}% Test data {pct_test*100}% \n')

    info_train, info_prune = do_train(pct_test)
    
    df_info_train = add_info(info_train, 1 - pct_test, df_info_train)
    df_info_prune = add_info(info_prune, 1 - pct_test, df_info_prune)

    # save(df_info_train, 'train', pct_data)
    # save(df_info_prune, 'prune', pct_data)

    # df_info.plot.scatter(x='percentage', y='accuracy')
    return df_info_train, df_info_prune    

In [7]:
def on_pct_data(p, ptest):
    global x, y, dft, dfp
    x, y = full_pct_data(p, ptest)
    x['epochs'] = EPOCHS
    x['pct_data'] = p
    x['pct_test'] = ptest

    y['epochs'] = EPOCHS_PRUNE
    y['pct_data'] = p
    y['pct_test'] = ptest

    dft = dft.append(x, ignore_index=True, verify_integrity=True) #trained
    dfp = dfp.append(y, ignore_index=True, verify_integrity=True)

    print()
    display('Trained', x)
    display('Pruned', y)
    print()
    print('Training Acc. Diff', (x.accuracy[0] - x.val_accuracy[0])*100)
    print('Pruned Acc. Diff', (y.accuracy[0] - y.val_accuracy[0])*100)
    # display('Difference', x - y)

In [8]:
dft = pd.DataFrame()
dfp = pd.DataFrame()

In [9]:
EPOCHS = 30
EPOCHS_PRUNE = 30
VERBOSE = 1
SHOW = 0

In [16]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

model.compile(optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

# history = model.fit(train_images, train_labels, epochs=EPOCHS, 
#                     validation_data=(test_images, test_labels), verbose=VERBOSE)

In [17]:
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity.keras import prune_low_magnitude, ConstantSparsity

pruning_params = {
    'pruning_schedule': ConstantSparsity(0.1, 0), #target, begin step
    'block_size': (1, 1),
    'block_pooling_type': 'AVG'}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

Instructions for updating:
Please use `layer.add_weight` method instead.


In [20]:
model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d ( (None, 30, 30, 32)        1762      
_________________________________________________________________
prune_low_magnitude_max_pool (None, 15, 15, 32)        1         
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 13, 13, 64)        36930     
_________________________________________________________________
prune_low_magnitude_max_pool (None, 6, 6, 64)          1         
_________________________________________________________________
prune_low_magnitude_conv2d_2 (None, 4, 4, 64)          73794     
_________________________________________________________________
prune_low_magnitude_flatten  (None, 1024)              1         
_________________________________________________________________
prune_low_magnitude_dense (P (None, 64)                1

In [21]:
log_dir = tempfile.mkdtemp()

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    # Log sparsity and other metrics in Tensorboard.
    tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir)]

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [24]:
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=15, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/15
36/36 [==============================] - 1s 21ms/step - loss: 1.7701 - accuracy: 0.3391 - val_loss: 1.7272 - val_accuracy: 0.3700
Epoch 2/15
36/36 [==============================] - 1s 20ms/step - loss: 1.6421 - accuracy: 0.4062 - val_loss: 1.7365 - val_accuracy: 0.3540
Epoch 3/15
36/36 [==============================] - 1s 20ms/step - loss: 1.5194 - accuracy: 0.4496 - val_loss: 1.7674 - val_accuracy: 0.3840
Epoch 4/15
36/36 [==============================] - 1s 21ms/step - loss: 1.4551 - accuracy: 0.4698 - val_loss: 1.6857 - val_accuracy: 0.4100
Epoch 5/15
36/36 [==============================] - 1s 21ms/step - loss: 1.3262 - accuracy: 0.5213 - val_loss: 1.7165 - val_accuracy: 0.4000
Epoch 6/15
36/36 [==============================] - 1s 21ms/step - loss: 1.2283 - accuracy: 0.5651 - val_loss: 1.6776 - val_accuracy: 0.4200
Epoch 7/15
36/36 [==============================] - 1s 21ms/step - loss: 1.1223 - accuracy: 0.5891 - val_loss: 1.8025 - val_accuracy: 0.3980
Epoch 8/15
36

In [ ]:
change_pct_data(1)
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=15, validation_split=validation_split,
                  callbacks=callbacks)